[Data Link: ](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-texts)

In [ ]:
pip install

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
data_path = "/content/data/test.csv"

In [6]:
df = pd.read_csv(data_path)
df.head()

,text,sentiment
0,"My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn't awful enough to see ""product placement"" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: wa...",neg
1,I... No words. No words can describe this. I will try for the sake of those few brave people who stick knives into their toasters... after watching this show.<br /><br />This... Cosgrove person... Her acting is like watching a female gorilla dance upo...,neg
2,"this film is basically a poor take on the old urban legend of the babysitter who gets crank calls telling her to check the children, she calls the police who trace the calls and find there coming from inside the house. when a killer calls has a story ...",neg
3,"This is a terrible movie, and I'm not even sure why it's so terrible. It's ugly, for one, with that trendy 1970s visual style that maybe seemed like a good idea at the time but which now enables one to instantly recognize a film from that time period ...",neg
4,"First of all this movie is a piece of reality very well realized artistically. Some kind of combination between ""American Beauty"" and ""As Good as it gets"". And of course something specifically to all Russian movies ( of course the valuable one, no dir...",pos


In [7]:
df.shape

(25000, 2)

In [8]:
df = df.iloc[:10000]
df.head()

,text,sentiment
0,"My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn't awful enough to see ""product placement"" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: wa...",neg
1,I... No words. No words can describe this. I will try for the sake of those few brave people who stick knives into their toasters... after watching this show.<br /><br />This... Cosgrove person... Her acting is like watching a female gorilla dance upo...,neg
2,"this film is basically a poor take on the old urban legend of the babysitter who gets crank calls telling her to check the children, she calls the police who trace the calls and find there coming from inside the house. when a killer calls has a story ...",neg
3,"This is a terrible movie, and I'm not even sure why it's so terrible. It's ugly, for one, with that trendy 1970s visual style that maybe seemed like a good idea at the time but which now enables one to instantly recognize a film from that time period ...",neg
4,"First of all this movie is a piece of reality very well realized artistically. Some kind of combination between ""American Beauty"" and ""As Good as it gets"". And of course something specifically to all Russian movies ( of course the valuable one, no dir...",pos


In [9]:
df.shape

(10000, 2)

In [10]:
df['text'][0]

'My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn\'t awful enough to see "product placement" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: was that intentional? Did big tobacco "sponsor" the film? What does it take to ban smoking from films? At least films intended for children and adolescents. My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn\'t awful enough to see "product placement" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: was that intentional? Did big tobacco "sponsor" the film? What does it take to ban smoking from films? At least films intended for children and adolescents.'

In [11]:
df['sentiment'].value_counts()

,count
sentiment,
neg,5021
pos,4979


In [12]:
df.isnull().sum()

,0
text,0
sentiment,0


In [13]:
df.duplicated().sum()

28

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
df.duplicated().sum()

0

# Basic Preprocessing
  - Remove tags - HTML
  - Lower case
  - remove stopwords

In [16]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [18]:
df['text'] = df['text'].apply(remove_tags)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [19]:
df['text'][0]

'My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn\'t awful enough to see "product placement" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: was that intentional? Did big tobacco "sponsor" the film? What does it take to ban smoking from films? At least films intended for children and adolescents. My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn\'t awful enough to see "product placement" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: was that intentional? Did big tobacco "sponsor" the film? What does it take to ban smoking from films? At least films intended for children and adolescents.'

In [20]:
df['text'] = df['text'].apply(lambda x:x.lower())

In [21]:
df['text'][0]

'my daughter liked it but i was aghast, that a character in this movie smokes. as if it isn\'t awful enough to see "product placement" actors like bruce willis who smoke in their movies - at least children movies should be more considerate! i wonder: was that intentional? did big tobacco "sponsor" the film? what does it take to ban smoking from films? at least films intended for children and adolescents. my daughter liked it but i was aghast, that a character in this movie smokes. as if it isn\'t awful enough to see "product placement" actors like bruce willis who smoke in their movies - at least children movies should be more considerate! i wonder: was that intentional? did big tobacco "sponsor" the film? what does it take to ban smoking from films? at least films intended for children and adolescents.'

In [22]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
sw_list = stopwords.words('english')

df['text'] = df['text'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
df['text'][0]

'daughter liked aghast, character movie smokes. awful enough see "product placement" actors like bruce willis smoke movies - least children movies considerate! wonder: intentional? big tobacco "sponsor" film? take ban smoking films? least films intended children adolescents. daughter liked aghast, character movie smokes. awful enough see "product placement" actors like bruce willis smoke movies - least children movies considerate! wonder: intentional? big tobacco "sponsor" film? take ban smoking films? least films intended children adolescents.'

In [24]:
df.head()

,text,sentiment
0,"daughter liked aghast, character movie smokes. awful enough see ""product placement"" actors like bruce willis smoke movies - least children movies considerate! wonder: intentional? big tobacco ""sponsor"" film? take ban smoking films? least films intende...",neg
1,"i... words. words describe this. try sake brave people stick knives toasters... watching show.this... cosgrove person... acting like watching female gorilla dance upon nest highly agitated insects. perhaps exaggerate. however, feeling description fits...",neg
2,"film basically poor take old urban legend babysitter gets crank calls telling check children, calls police trace calls find coming inside house. killer calls story simplistic little kid could written it. much suspense, becomes clear killer halfway fil...",neg
3,"terrible movie, i'm even sure terrible. ugly, one, trendy 1970s visual style maybe seemed like good idea time enables one instantly recognize film time period 70s product. film retains story songs made stage version musical hit, songs sound lifeless s...",neg
4,"first movie piece reality well realized artistically. kind combination ""american beauty"" ""as good gets"". course something specifically russian movies ( course valuable one, dirty propaganda !) : problem loneliness man ... especially recommended people...",pos


In [25]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [26]:
X.head()

,text
0,"daughter liked aghast, character movie smokes. awful enough see ""product placement"" actors like bruce willis smoke movies - least children movies considerate! wonder: intentional? big tobacco ""sponsor"" film? take ban smoking films? least films intende..."
1,"i... words. words describe this. try sake brave people stick knives toasters... watching show.this... cosgrove person... acting like watching female gorilla dance upon nest highly agitated insects. perhaps exaggerate. however, feeling description fits..."
2,"film basically poor take old urban legend babysitter gets crank calls telling check children, calls police trace calls find coming inside house. killer calls story simplistic little kid could written it. much suspense, becomes clear killer halfway fil..."
3,"terrible movie, i'm even sure terrible. ugly, one, trendy 1970s visual style maybe seemed like good idea time enables one instantly recognize film time period 70s product. film retains story songs made stage version musical hit, songs sound lifeless s..."
4,"first movie piece reality well realized artistically. kind combination ""american beauty"" ""as good gets"". course something specifically russian movies ( course valuable one, dirty propaganda !) : problem loneliness man ... especially recommended people..."


In [27]:
y

,sentiment
0,neg
1,neg
2,neg
3,neg
4,pos
5,neg
6,neg
7,neg
8,pos
9,neg


In [28]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [29]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [31]:
X_train.shape

(7977, 1)

In [32]:
X_test.shape

(1995, 1)

In [33]:
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [34]:
X_train_bow = cv.fit_transform(X_train['text']).toarray()
X_test_bow = cv.transform(X_test['text']).toarray()

In [35]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [36]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [37]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.687218045112782

In [38]:
confusion_matrix(y_test,y_pred)

array([[804, 248],
       [376, 567]])

In [40]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)

y_pred = rf.predict(X_test_bow)

accuracy_score(y_test,y_pred)

0.843609022556391

In [41]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['text']).toarray()
X_test_bow = cv.transform(X_test['text']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8280701754385965

# N grams

In [42]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['text']).toarray()
X_test_bow = cv.transform(X_test['text']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8370927318295739

TFIDF

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [44]:
X_train_tfidf = tfidf.fit_transform(X_train['text']).toarray()
X_test_tfidf = tfidf.transform(X_test['text'])

In [45]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8365914786967419